In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import ee
import geemap
import time

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')
print("GEE Authenticated !")

GEE Authenticated !


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
def monitor_export(task):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish..\n\tPatience young padawan.')
        time.sleep(30)  # Check every 30 seconds
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully !!!!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Bummer. Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions loaded !")

Functions loaded !


In [3]:
# Load the gridmet image collection

In [4]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}")

gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']


In [5]:
# calculate the 15 year average
mean15 = gridmet.filter(ee.Filter.calendarRange(2009,2024,'year')).mean() 
gridmet.first().bandNames().getInfo()

['pr',
 'rmax',
 'rmin',
 'sph',
 'srad',
 'th',
 'tmmn',
 'tmmx',
 'vs',
 'erc',
 'eto',
 'bi',
 'fm100',
 'fm1000',
 'etr',
 'vpd']

In [6]:
# Load the AFD 

In [7]:
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/vnp14img_geo_srm_pix_area_aspenfires')
print(f"Number of AFD: {afds.size().getInfo()}")
print(afds.first().propertyNames().getInfo())

Number of AFD: 51853
['Fire_ID', 'acq_date', 'daynight', 'Ig_Date', 'Last_Date', 'afdID', 'system:index']


In [8]:
# get a list of unique active fire days
fire_days = afds.aggregate_array('acq_date').getInfo()
fire_days = set(fire_days)
print(f"Number of 'fire days': {len(fire_days)}")

Number of 'fire days': 390


In [9]:
# map over the list of unique dates
def gridmet_summary(obs, attrs):
    """ calculate the daily gridmet for active fire detections """
    fire_days = obs.aggregate_array('acq_date').getInfo()
    fire_days = set(fire_days)
    
    def get_daily(day):
        # filter obs. to the day of burning
        afds_ = obs.filter(ee.Filter.eq('acq_date',day))
        acq_date = ee.Date(day) # the day of acquisition
        prev_day = acq_date.advance(-1,'days') # previous day for calculating the mean

        bounds = afds_.geometry().bounds() # bounds for reduction
        
        # calculate the gridmet mean
        gridmet_ = gridmet.select(attrs).filterBounds(bounds)
        gridmet_ = gridmet_.filterDate(prev_day, acq_date) # filter to afd bounds
        gridmet_ = gridmet_.mean() # the average prev day to day of burn
        gridmet_mn = gridmet_.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=bounds,
            scale=4000,
            maxPixels=1e13
        )

        # calculation the deviation from the 15-year average
        mean15_ = mean15.select(attrs)
        anomaly = gridmet_.subtract(mean15_)
        anomaly_mn = anomaly.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=bounds,
            scale=4000,
            maxPixels=1e13
        )

        return ee.Dictionary({
            'acq_date': day,
            'daily_mean': gridmet_mn,
            'anomaly': anomaly_mn
        })

    daily_stats = ee.FeatureCollection([
        ee.Feature(None, get_daily(day))
        for day in fire_days
    ])

    return daily_stats

print("Function ready !")

Function ready !


In [10]:
# list of gridmet attributes
attrs = ['fm1000', 'vpd', 'erc', 'bi', 'tmmx', 'rmin', 'vs']

fire_ids = afds.aggregate_array('Fire_ID').getInfo()
fire_ids = set(fire_ids)

# loop over fire ids
def process_fires(afds, fire_ids, attrs):
    results = []
    for fire_id in fire_ids:
    
        afds_ = afds.filter(ee.Filter.eq('Fire_ID',fire_id))
    
        # calculate gridmet for the active fire detections
        fire_fc = gridmet_summary(afds_, attrs)
        results.append(fire_fc)
    
    results_fc = ee.FeatureCollection(results).flatten()
    return results_fc

afds_gridmet = process_fires(afds, fire_ids, attrs)
# print("Number of features:", afds_gridmet.size().getInfo())
print("Submitted !")

Submitted !


In [11]:
# Export the table. 
afds_gridmet = afds_gridmet.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=afds_gridmet,
    description='vn14img_gridmet',
    fileFormat='CSV', 
    fileNamePrefix='vn14img_gridmet',
    folder='GRIDMET'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) # print every 2 min

Export to Earth Engine Asset started!


NameError: name 'monitor_gee_export' is not defined